In [1]:
import pandas as ps
import rasterio
import re
import numpy as np

from shapely.geometry import Polygon
from shapely.geometry import shape

import rasterio
from rasterio.mask import mask
from rasterio.transform import from_bounds
from rasterio.plot import show, show_hist
from rasterio.fill import fillnodata

import os
import shapely
import geojson
import fiona

In [2]:
# base_dir = r"D:\Acadamics\University\Year_3\Sem_2\GIS3005 - Remote Sensing\Assignment\1"
base_dir = os.getcwd()
countries_interested = ["Pakistan", "Nepal", "Sri Lanka", "India", "Bhutan", "Bangladesh"]
gdp_dataset = os.path.join(base_dir, "GDP", "API_NY.GDP.MKTP.CD_DS2_en_csv_v2_4770391.csv")
gdp_dataset_filtered = os.path.join(base_dir, "output", "API_NY.GDP.MKTP.CD_DS2_en_csv_v2_4770391_filtered.csv")
nightlight = os.path.join(base_dir, "output", "nightlight.csv")
output_folder = os.path.join(base_dir, r"output")
year_from = 1992
year_to = 2013

In [28]:
'''
Filter WorldBank's current GDP dataset
'''
df = ps.read_csv(gdp_dataset, skiprows=0, na_values=None)
df.drop(['Unnamed: 66'], axis=1, inplace=True)
df.rename(columns={'Country Name':'Name', 'Country Code':'Code', 'Indicator Name':'IName', 'Indicator Code':'ICode'}, inplace=True)
# print(df.head())

df = df.drop([str(x) for x in range(1960, 1992)] + [str(x) for x in range(2014, 2022)] + ['IName', 'ICode'], axis=1)
df = df[df.Name.isin(countries_interested)]
# df = df.set_index('Name')
# print(df.head())

df.to_csv(gdp_dataset_filtered, index=True)


           Name Code          1992          1993          1994          1995   
20   Bangladesh  BGD  3.170887e+10  3.316652e+10  3.376866e+10  3.793975e+10  \
32       Bhutan  BTN  2.402158e+08  2.259981e+08  2.589856e+08  2.904648e+08   
109       India  IND  2.882084e+11  2.792960e+11  3.272756e+11  3.602820e+11   
138   Sri Lanka  LKA  9.703012e+09  1.033868e+10  1.171760e+10  1.302970e+10   
178       Nepal  NPL  3.401212e+09  3.660042e+09  4.066776e+09  4.401104e+09   

             1996          1997          1998          1999  ...   
20   4.643848e+10  4.824431e+10  4.998456e+10  5.127057e+10  ...  \
32   3.034355e+08  3.522610e+08  3.634528e+08  3.992688e+08  ...   
109  3.928971e+11  4.158678e+11  4.213515e+11  4.588204e+11  ...   
138  1.389774e+10  1.509191e+10  1.579497e+10  1.565633e+10  ...   
178  4.521580e+09  4.918692e+09  4.856255e+09  5.033642e+09  ...   

             2004          2005          2006          2007          2008   
20   6.510854e+10  6.944294e+10  

In [4]:
class progress():
    def __init__(self, current, maximum):
        self.current = current
        self.maximum = maximum
        self.BAR_WIDTH = 40
        
        self.current -=1
        self.next()
        
        
    def next(self):
        self.current += 1
        x = int(self.BAR_WIDTH*self.current/self.maximum)
        y = round(self.current/self.maximum*100, 1)
        text_pb = "{}[{}{}] {}/{} {}%".format("Processing", "#"*x, "."*(self.BAR_WIDTH-x), self.current, self.maximum, y)
        print(text_pb, end='\r', file=sys.stdout, flush=True)

In [5]:
def get_files(path, extention=".tif", regex=""):
    files = []
    for file in os.listdir(path):
        if file.endswith(extention):
            if regex != "":
                if not re.match(regex, file): continue
            temp_path = os.path.join(path, file)
            if os.path.exists(temp_path):
                files.append(temp_path)
    return files

In [6]:
def load(npy_file, band=None):
    if os.path.isfile(npy_file):
        if band == None: return np.load(npy_file)
        return np.load(npy_file)[band]
    return None

In [7]:
def clip_(raster, geometry):
    out_image = mask(raster, geometry, crop=True)
    return out_image

In [82]:
def night_light_development_index(data):

    nldi = (data - np.mean(data)) / np.std(data)
    return np.mean(nldi)

def nighttime_lights_urbanization_index(data):

    nlui = np.max(data) / np.sum(data)
    return nlui

def nighttime_lights_density_index(data):
    
    nldi = np.sum(data) / np.prod(data.shape)
    return nldi

In [70]:
def calculate_night_light_development_index(array):
    # Normalize array between 0 and 1
    normalized_array = (array - np.min(array)) / (np.max(array) - np.min(array))
    nldi = np.mean(normalized_array)
    
    return nldi

def calculate_nighttime_lights_urbanization_index(array):
    # Calculate NLUI
    # nlui = np.sum(array > np.mean(array)) / array.size
    nlui = np.sum(array) / np.max(array)
    # nlui = np.sum(array) / array.size
    
    return nlui

def calculate_nighttime_lights_density_index(array):
    # Calculate NLDI
    # nldi = np.sum(array) / array.size
    nldi = np.sum(array) / np.sum(array > 0)
    # nldi = np.sum(array) / (np.max(array) * array.size)
    
    return nldi


In [30]:
def NLUIndex(npy):
    data = npy
    # Calculate the mean brightness of artificial lights in urban areas.
    mean_brightness_urban = np.mean(data[data > 0])
    # Calculate the mean brightness of artificial lights in rural areas.
    mean_brightness_rural = np.mean(data[data <= 0])
    # Calculate the NLUL.
    nlui = mean_brightness_urban / mean_brightness_rural
    return nlui
def NLDIndex(npy):
    data = npy
    # Calculate the mean brightness of artificial lights at night.
    mean_brightness = np.mean(data)

    # Calculate the area of the land.
    area = data.shape[0] * data.shape[1]

    # Calculate the NLDI.
    nldi = mean_brightness / area
    return nldi

In [10]:
def clip(shapefile_path, raster_path, clipped_folder="out", attribute='NAME_0'):
    # Open the shapefile
    with fiona.open(shapefile_path, "r") as shapefile:
        # Iterate over the features
        for feature in shapefile:
            # Get the geometry
            geometry = shape(feature["geometry"])
            name_suffix = feature['properties'][attribute]
            year = str(re.findall(r"F\d\d(\d+)", raster_path)[0])
            # print(shapefile.schema['properties'].keys())

            # Open the raster file
            with rasterio.open(raster_path) as raster:
                # Clip the raster using the current geometry
                clipped, transform = mask(dataset=raster, shapes=[geometry], crop=True)

                # Update the metadata for the clipped raster
                clipped_meta = raster.meta.copy()
                clipped_meta.update({
                    "width": clipped.shape[2],
                    "height": clipped.shape[1],
                    "transform": transform
                })

                # Generate a unique filename for the clipped raster
                filename = f"{name_suffix}_{year}.tif"
                clipped_path = f"{clipped_folder}/{filename}"

                # Write the clipped raster to a new file
                with rasterio.open(clipped_path, "w", **clipped_meta) as dst:
                    dst.write(clipped)

In [11]:
def process_non_gee(tif_collection):
    print("Processing...", end="")
    shp = os.path.join(base_dir, "sAsia", "SAsia_Merged.shp")
    for tif in tif_collection:
        year = str(re.findall(r"F\d\d(\d+)", tif)[0])
        print(year + ", ", end="")
        clip(shp, tif, "output")

In [77]:
'''
Process Night Light Entensity npy
'''
gee = True # is this data from gee

if gee:
    npys = get_files(output_folder, '.npy')
    npys = [{'npy':load(x)['b1'], 'country':os.path.splitext(os.path.split(x)[-1])[0].split("_")[0], 'year':int(os.path.splitext(os.path.split(x)[-1])[0].split("_")[-1])} for x in npys]
else:
    tif = get_files(output_folder, '.tif')
    process_non_gee(tif)
        


In [78]:
df = ps.DataFrame(npys)

In [79]:
data = {"country":[], "year":[], 'nldi':[], 'nlui':[]}
for country in countries_interested:
    nldf = df[df.country == country]
    for index, item in nldf.iterrows():
        b1 = item.npy
        data["country"].append(country)
        data["year"].append(item.year)
        # data["nldi"].append(calculate_night_light_development_index(b1))
        data["nldi"].append(nighttime_lights_density_index(b1))
        
        data["nlui"].append(calculate_nighttime_lights_urbanization_index(b1))

In [80]:
df = ps.DataFrame(data)
# df.set_index('country')

In [81]:
df.to_csv(nightlight, index=True)